In [1]:
import xarray as xr
import numpy as np
import datetime
def data_ones(x):
    x_new=x/x
    x_new=x_new.fillna(0)
    return x_new
dir_data = 'F:/data/sat_data/oscar/L4/oscar_third_deg/'

In [4]:
#make oscar monthly data, change latitue and longitude to lat and lon, group by month
#dir_data = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/'
for lyr in range(1993,2019): #2017):
    filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc'
    ds=xr.open_dataset(filename,drop_variables=['um','vm'])
    ds_month = ds.resample(time='M').mean()    
    ds_month.rename({'longitude':'lon','latitude':'lat'}, inplace = True)
    ds.close()
    ds_month.to_netcdf(dir_data + str(lyr) + 'monthly_data_oscar.nc')
#check that yes, resampling is averaging for center of month
#ds_month.u[0,0,450,900].data
#ds.u[0:7,0,450,900].mean()

C:\Users\gentemann\Miniconda3\lib\site-packages\xarray\core\nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
C:\Users\gentemann\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  
C:\Users\gentemann\Miniconda3\lib\site-packages\xarray\core\nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
C:\Users\gentemann\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  
C:\Users\gentemann\Miniconda3\lib\site-packages\xarray\core\nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
C:\Users\gentemann\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  
C:\Users\gentema

C:\Users\gentemann\Miniconda3\lib\site-packages\xarray\core\nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
C:\Users\gentemann\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.
  


In [42]:
#make monthly climatology, need to drop 'year' coordinate value for mean to work right
for lyr in range(1993,2019):
    filename = dir_data + str(lyr) + 'monthly_data_oscar.nc'
    ds=xr.open_dataset(filename)
    ds2 = ds.drop('year')
    if lyr==1993:
        ds_clim = ds2
    else:
        ds_clim = xr.concat([ds_clim,ds2],dim = 'time')
    ds.close()
ds_clim2 = ds_clim.groupby('time.month').mean('time')
ds_clim2.to_netcdf(dir_data + 'climatology_1993_2018_monthly_data_oscar.nc')

C:\Users\gentemann\Miniconda3\lib\site-packages\xarray\core\nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [45]:
ds_clim2.to_netcdf(dir_data + 'climatology_1993_2018_monthly_data_oscar.nc')

In [ ]:
#OLD OLD OLD OLD OLD OLD CODE
#make oscar climatology, OLD version of code
#dir_data='F:/data/sat_data/oscar/L4/oscar_third_deg/'
dir_clim='F:/data/sat_data/oscar/L4/oscar_third_deg/climatology/'
dir_data = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/'
for lyr in range(1993,2018):
    filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc.gz'
    ds=xr.open_dataset(filename,drop_variables=['um','vm'])
    ds_count=data_ones(ds)
    #in order to add up the data the time arrays have to be aligned otherwise xarray doesn't know what to do with it
    ds['time'] = ds['time'] - np.datetime64(lyr,'Y')
    ds_count['time'] = ds_count['time'] - np.datetime64(lyr,'Y')
    if lyr==1993:
        ds2=ds.fillna(0)
        ds_count2=ds_count.fillna(0)
        ds_summer=ds2
        ds_counter = ds_count2
    else:
        ds2 = ds.interp_like(ds_summer.time)
        ds_count2 = ds_count.interp_like(ds_summer.time)
        ds2=ds2.fillna(0)
        ds_count2=ds_count2.fillna(0)
        ds_summer=ds_summer + ds2
        ds_counter = ds_counter + ds_count2
    print(lyr)
    print(ds.u.shape)
    print(ds_summer.u.shape)
   # print(ds_counter.u.shape)
   # print(ds.u.shape)
   # print(ds.time.data[0:20])
ds_ave = ds_summer / ds_counter
ds_ave.to_netcdf(dir_clim + 'oscar_v2009_1993_2016_climatology_12202118.nc')